# **CO2 emissions and climate change** 
### <br>The past, present and future responsibilities<br>  

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import pycountry_convert as pc
import plotly.express as px
import plotly.offline as pyo
import plotly.express as px
import plotly.io as pio
import numpy as np
#pyo.init_notebook_mode()

pio.renderers.default = 'notebook'

In [2]:
def country_to_continent(country_code):
    try:
        country_alpha2 = pc.country_alpha3_to_country_alpha2(country_code)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    except: 
        country_continent_name = country_code
        
    return country_continent_name

In [3]:
df = pd.read_csv('data/owid-co2-data.csv',sep = ',',encoding = 'unicode_escape')
df = df.fillna(0)
temp = df[['iso_code', 'country', 'year', 'co2','energy_per_gdp', 'co2_per_capita','primary_energy_consumption','gdp','co2_per_gdp']] 

In [21]:
df_sector = pd.read_csv('data/co-emissions-by-sector.csv',sep = ',',encoding = 'unicode_escape')
df_sector = df_sector.fillna(0)
df_sector['Continent'] = df_sector['Code'].apply(lambda x: country_to_continent(x))

In [10]:
len(set(df.country))

243

In [22]:
df_sector_18 = df_sector[(df_sector['Year'] == 2018) & ~(df_sector['Continent'].isin([0,'TLS','OWID_WRL']))]
df_sun = df_sector_18[~df_sector_18[['Buildings', 'Industry',
       'Land-use change and forestry', 'Other fuel combustion', 'Transport',
       'Manufacturing and construction', 'Fugitive emissions',
       'Electricity and heat']].eq(0).all(axis = 1)].drop(columns = ["Year","Code","Land-use change and forestry"])

In [1]:
df_sun2 = df_sun.melt(id_vars=["Entity","Continent"], 
        var_name="Sector", 
        value_name="Value")
#temp = df_sun2[df_sun2['Entity'].isin(['Mongolia'])]
#temp
fig = px.sunburst(df_sun2, path=['Continent', 'Entity','Sector'], values='Value', color='Continent',width=1000, height=800)
fig.update_traces(textinfo="label+percent entry ")
fig.show()

NameError: name 'df_sun' is not defined

In [12]:
mask = (temp['year'] > 1990) & (temp['country'].isin(['Asia','Europe','Oceania','Africa',
                                                      'North America','South America','Antarctica']))

mask2 = (temp['year'] > 1990) & ~(temp['country'].isin(['Asia','Europe','Oceania','Africa',
                                                      'North America','South America','Antarctica','World']))

mask3 = (temp['year'] > 1990) & (temp['country'].isin(['China','United States','Canada','Qatar',
                                                      'India','Germany','United Kingdom','Saudi Arabia']))

In [13]:
cont_co2 = temp[mask2]
cont_co22 = temp[mask]

In [14]:
fig = px.choropleth(cont_co2, locations="iso_code", color="co2", hover_name="country", animation_frame="year",
                    range_color=[np.percentile(cont_co2['co2'],5),max(cont_co2['co2'])],
                    projection = 'natural earth',
                    title="Total CO2 emission per country from 1990 to 2020",labels={'co2':'million tonnes'})
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 150
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 10
fig.show()


In [15]:
fig = px.choropleth(cont_co2, locations="iso_code", color="co2_per_capita", hover_name="country", animation_frame="year",
                    range_color=[np.percentile(cont_co2['co2_per_capita'],1),max(cont_co2['co2_per_capita'])],
                    projection='natural earth',
                    title="CO2 emission per capita per country from 1990 to 2020",labels={'co2_per_capita':'tonnes'})
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 150
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 10
fig.show()

In [154]:
temp = cont_co2[cont_co2['country'].isin(['China','India','United States','Germany','Saudi Arabia'])]
fig = px.scatter(temp, x="primary_energy_consumption", y="co2", color='country', trendline="ols")
fig.show()

In [146]:
df2 = px.data.gapminder()
fig = px.scatter(temp, x="primary_energy_consumption", y="co2", animation_frame="year", animation_group="country",
           size="co2_per_capita", color="country", hover_name="country", facet_col="country",labels={'country':'Country'},
           log_x=False, log_y= False, size_max=45, range_x=[1,max(temp['energy_per_gdp'])], range_y=[1,max(temp['co2_per_capita'])])

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 150
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 10
fig.show()

In [156]:
df.describe()

,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,flaring_co2,flaring_co2_per_capita,...,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
count,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,...,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,2.519100e+04,2.519100e+04,25191.000000,25191.000000,25191.000000
mean,1953.195546,254.655458,3.858882,-0.381268,6.266510,0.054825,119.648138,1.038412,2.609322,0.152228,...,1.541270,17.750555,0.427005,6.529960,0.135134,6.422993e+07,1.546188e+11,541.278196,8811.084895,0.499684
std,53.788944,1484.826244,14.385611,72.480032,54.253863,0.118091,654.445808,2.211787,17.810338,2.308195,...,4.479964,260.308638,1.761079,94.657017,0.490859,3.623151e+08,1.604593e+12,5376.643859,23026.452190,1.159689
min,1750.000000,0.000000,0.000000,-1657.998000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,1924.000000,0.359000,0.152000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.873210e+05,0.000000e+00,0.000000,0.000000,0.000000
50%,1967.000000,3.972000,1.001000,0.000000,0.000000,0.000000,0.395000,0.055000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.008723e+06,2.164603e+09,0.000000,0.000000,0.000000
75%,1995.000000,38.419500,4.257000,0.000000,0.510000,0.066000,12.142500,1.008500,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.478986e+07,3.639273e+10,9.299000,3910.669000,0.427500
max,2020.000000,36702.503000,748.639000,1028.487000,1626.371000,2.738000,15062.902000,34.184000,435.034000,94.711000,...,53.650000,8298.270000,39.795000,3078.270000,10.056000,7.794799e+09,1.136302e+14,162194.290000,317582.498000,13.493000
